In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torch.nn as nn
import copy
import time
import os
import argparse
from google.colab import files
from resnet import *
from torch.utils.data import Subset
import csv

In [3]:
device = torch.device('cuda')

In [4]:
transform_test = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

transform_eval = transforms.Compose([
        transforms.RandomResizedCrop((32,32), scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])


In [5]:
dataset = torchvision.datasets.CIFAR10(
      root='./data', train=True, download=True, transform= transform_eval)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
#Create a subset from training data for evaluation -- Due to computational resource limitations, I have used 10000 images for training. 

with open('evalset.txt', 'w',newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['current_class','label'])

def create_evalset(ds):
  
  airplane_i, dog_i, deer_i, frog_i, cat_i, bird_i, horse_i, automobile_i, ship_i, truck_i = [], [], [], [], [],[],[],[],[],[]
  dog_idx, deer_idx, airplane_idx, frog_idx, cat_idx, bird_idx, automobile_idx, ship_idx, truck_idx, horse_idx = ds.class_to_idx['dog'], ds.class_to_idx['deer'],ds.class_to_idx['airplane'],ds.class_to_idx['frog'],ds.class_to_idx['cat'],ds.class_to_idx['bird'],ds.class_to_idx['automobile'],ds.class_to_idx['ship'],ds.class_to_idx['truck'],ds.class_to_idx['horse']
  label=[]
  for i in range(len(ds)):
    current_class = ds[i][1]
    if current_class == dog_idx:
      dog_i.append(i)
      label='dog'
    elif current_class == deer_idx:
      deer_i.append(i)
      label='deer'
    elif current_class == airplane_idx:
      airplane_i.append(i)
      label='airplane'
    elif current_class == frog_idx:
      frog_i.append(i)
      label='frog'
    elif current_class == cat_idx:
      cat_i.append(i)
      label='cat'
    elif current_class == bird_idx:
      bird_i.append(i)
      label='bird'
    elif current_class == automobile_idx:
      automobile_i.append(i)
      label='automobile'
    elif current_class == ship_idx:
      ship_i.append(i)
      label='ship'
    elif current_class == truck_idx:
      truck_i.append(i) 
      label='truck'
    else:
      horse_i.append(i)
      label='horse'
    with open('evalset.txt', 'a', newline='') as csvfile:
          writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
          writer.writerow([current_class, label])
# For 1% labelled data
  airplane_ei = airplane_i[int(0.999 * len(airplane_i)):]
  dog_ei = dog_i[int(0.999 * len(dog_i)):]
  deer_ei = deer_i[int(0.999 * len(deer_i)):]
  frog_ei = frog_i[int(0.999 * len(frog_i)):]
  cat_ei = cat_i[int(0.999 * len(cat_i)):]
  bird_ei = bird_i[int(0.999 * len(bird_i)):]
  horse_ei = horse_i[int(0.999 * len(horse_i)):]
  automobile_ei = automobile_i[int(0.999 * len(automobile_i)):]
  ship_ei = ship_i[int(0.999 * len(ship_i)):]
  truck_ei = truck_i[int(0.999 * len(truck_i)):]

# # For 10% labelled data
#   airplane_ei = airplane_i[int(0.98 * len(airplane_i)):]
#   dog_ei = dog_i[int(0.98 * len(dog_i)):]
#   deer_ei = deer_i[int(0.98 * len(deer_i)):]
#   frog_ei = frog_i[int(0.98 * len(frog_i)):]
#   cat_ei = cat_i[int(0.98 * len(cat_i)):]
#   bird_ei = bird_i[int(0.98 * len(bird_i)):]
#   horse_ei = horse_i[int(0.98 * len(horse_i)):]
#   automobile_ei = automobile_i[int(0.98 * len(automobile_i)):]
#   ship_ei = ship_i[int(0.98 * len(ship_i)):]
#   truck_ei = truck_i[int(0.98 * len(truck_i)):]


# # For 30% labelled data
#   airplane_ei = airplane_i[int(0.94 * len(airplane_i)):]
#   dog_ei = dog_i[int(0.94 * len(dog_i)):]
#   deer_ei = deer_i[int(0.94 * len(deer_i)):]
#   frog_ei = frog_i[int(0.94 * len(frog_i)):]
#   cat_ei = cat_i[int(0.94 * len(cat_i)):]
#   bird_ei = bird_i[int(0.94 * len(bird_i)):]
#   horse_ei = horse_i[int(0.94 * len(horse_i)):]
#   automobile_ei = automobile_i[int(0.94 * len(automobile_i)):]
#   ship_ei = ship_i[int(0.94 * len(ship_i)):]
#   truck_ei = truck_i[int(0.94 * len(truck_i)):]

# # For 50% labelled data
#   airplane_ei = airplane_i[int(0.9 * len(airplane_i)):]
#   dog_ei = dog_i[int(0.9 * len(dog_i)):]
#   deer_ei = deer_i[int(0.9 * len(deer_i)):]
#   frog_ei = frog_i[int(0.9 * len(frog_i)):]
#   cat_ei = cat_i[int(0.9 * len(cat_i)):]
#   bird_ei = bird_i[int(0.9 * len(bird_i)):]
#   horse_ei = horse_i[int(0.9 * len(horse_i)):]
#   automobile_ei = automobile_i[int(0.9 * len(automobile_i)):]
#   ship_ei = ship_i[int(0.9 * len(ship_i)):]
#   truck_ei = truck_i[int(0.9 * len(truck_i)):]

  evalset_l = Subset(ds, airplane_ei+ dog_ei+ deer_ei+ frog_ei+ cat_ei+ bird_ei+ horse_ei+ automobile_ei+ ship_ei+ truck_ei)

  
  return evalset_l
# files.download('evalset.txt')
evalset_l = create_evalset(dataset)

In [7]:
len(evalset_l)

50

In [8]:
evalloader = torch.utils.data.DataLoader(
    evalset_l, batch_size=32, shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=32, shuffle=True, num_workers=0)


In [9]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [10]:
class SimCLR_model(nn.Module):
    """
    We opt for simplicity and adopt the commonly used ResNet (He et al., 2016) to obtain hi = f(x ̃i) = ResNet(x ̃i) where hi ∈ Rd is the output after the average pooling layer.
    """

    def __init__(self, base_encoder, projection_dim, n_features):
        super(SimCLR_model, self).__init__()

        self.base_encoder = base_encoder
        self.n_features = n_features
        self.base_encoder.linear=nn.Linear(n_features, n_features)

        self.base_encoder.fc = Identity()
        self.projectionHead = nn.Sequential(
            nn.Linear(self.n_features, self.n_features, bias=False),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(), #non-linearity
            nn.Linear(self.n_features, projection_dim, bias=False),
            nn.BatchNorm1d(projection_dim),
        )

    def forward(self, x_i, x_j):
        h_i = self.base_encoder(x_i)
        h_j = self.base_encoder(x_j)
        
        z_i = self.projectionHead(h_i)
        z_j = self.projectionHead(h_j)
        
        return h_i, h_j, z_i, z_j

In [11]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [36]:
base_encoder = ResNet50()

projection_dim=128
n_features= base_encoder.linear.in_features
net= SimCLR_model(base_encoder, projection_dim, n_features)

net= net.to(device)
#net

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
base_encoder.linear.weight #weights of the linear layer before loading the model 

Parameter containing:
tensor([[-0.0144,  0.0113,  0.0193,  ...,  0.0185,  0.0148,  0.0106],
        [ 0.0090,  0.0129, -0.0217,  ...,  0.0117,  0.0032,  0.0162],
        [-0.0195,  0.0178,  0.0068,  ...,  0.0101, -0.0027, -0.0189],
        ...,
        [-0.0169,  0.0027, -0.0170,  ...,  0.0088,  0.0179,  0.0157],
        [-0.0022,  0.0221, -0.0049,  ..., -0.0192,  0.0219, -0.0141],
        [ 0.0045, -0.0160,  0.0067,  ..., -0.0118,  0.0014,  0.0177]],
       device='cuda:0', requires_grad=True)

In [37]:

path_paramsend = "/content/gdrive/MyDrive/paramsend_bs_restart.pt"
checkpoint = torch.load(path_paramsend, map_location=torch.device('cpu'))
loss= checkpoint['loss']
epoch = checkpoint['epoch']

start_epoch=0
net.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [38]:
epoch

375

In [39]:
net_eval= base_encoder
n_features= base_encoder.linear.in_features
# for param in net_eval.parameters():
#     param.requires_grad = False
net_eval.fc=nn.Linear(n_features, len(classes)) #change the fc layer with linear layer
# net_eval.to(device)

In [40]:
start_epoch=0
batch_size = 32
lr = 0.0001
device='cuda'
optimizer= torch.optim.SGD(net_eval.parameters(), lr, momentum=0.9, weight_decay = 0, nesterov=True)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
net_eval.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (

In [41]:
net_eval.linear.weight #weights of the linear layer after loading the model 

Parameter containing:
tensor([[ 0.0094, -0.0010, -0.0135,  ...,  0.0019, -0.0039, -0.0014],
        [-0.0129,  0.0169, -0.0109,  ...,  0.0089,  0.0045, -0.0145],
        [-0.0147,  0.0171,  0.0130,  ..., -0.0009, -0.0001,  0.0168],
        ...,
        [-0.0061, -0.0062, -0.0099,  ..., -0.0090,  0.0017, -0.0123],
        [ 0.0225, -0.0096, -0.0148,  ...,  0.0031,  0.0163, -0.0224],
        [-0.0138, -0.0134, -0.0107,  ...,  0.0139,  0.0035,  0.0249]],
       device='cuda:0', requires_grad=True)

In [42]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

def train(epoch):
    print('\nEpoch: %d' % epoch)
    net_eval.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(evalloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net_eval(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(f'{epoch} Loss: {train_loss:.4f} Acc: {correct/total:.4f}')

best_acc=0

def test(epoch):
    global best_acc
    net_eval.eval()
    test_loss = 0
    correct = 0
    total = 0
    acc=0
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net_eval(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            for label, prediction in zip(targets, predicted):
              if label == prediction:
                  correct_pred[classes[label]] += 1
              total_pred[classes[label]] += 1


        # print accuracy for each class
        for classname, correct_count in correct_pred.items():
            accuracy = 100 * float(correct_count) / total_pred[classname]
            print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

        print(f'{epoch} Loss: {test_loss/(batch_idx+1):.4f} Acc: {correct/total:.4f}')

    acc = 100.*correct/total
    print(acc)
    if acc > best_acc:
        # print('Saving..')
        parameters_finetune = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'loss': test_loss
        }

        # modelstate_save_name = 'modelstate_finetune.pt'
        # parameters_save_name = 'params_finetune.pt'
        # path_modelstate = F"/content/gdrive/MyDrive/{modelstate_save_name}" 
        # torch.save(net_eval.state_dict(), path_modelstate)
        # path_paramsfinetune = F"/content/gdrive/MyDrive/{parameters_save_name}" 
        # torch.save(parameters_finetune, path_paramsfinetune)
        best_acc = acc
    print("best_acc", best_acc)
for epoch in range(start_epoch, start_epoch+30):
    train(epoch)
    test(epoch)

    # if epoch %25==0:
    #   parameters_end = {
    #         'net': net_eval.state_dict(),
    #         'acc': acc,
    #         'epoch': epoch,
    #         'loss': test_loss
    #   }
      # paramsend_save_name = 'paramsend_finetune.pt'
      # path_paramsend = F"/content/gdrive/MyDrive/{paramsend_save_name}" 
      # torch.save(parameters_end, path_paramsend)


Epoch: 0
0 Loss: 16.7340 Acc: 0.0000
Accuracy for class: plane is 0.0 %
Accuracy for class: car   is 0.0 %
Accuracy for class: bird  is 0.0 %
Accuracy for class: cat   is 0.0 %
Accuracy for class: deer  is 0.0 %
Accuracy for class: dog   is 0.0 %
Accuracy for class: frog  is 0.0 %
Accuracy for class: horse is 0.0 %
Accuracy for class: ship  is 0.0 %
Accuracy for class: truck is 0.0 %
0 Loss: 8.5012 Acc: 0.0000
0.0
best_acc 0

Epoch: 1
1 Loss: 16.7642 Acc: 0.0000
Accuracy for class: plane is 0.0 %
Accuracy for class: car   is 0.0 %
Accuracy for class: bird  is 0.0 %
Accuracy for class: cat   is 0.0 %
Accuracy for class: deer  is 0.0 %
Accuracy for class: dog   is 0.0 %
Accuracy for class: frog  is 0.0 %
Accuracy for class: horse is 0.0 %
Accuracy for class: ship  is 0.0 %
Accuracy for class: truck is 0.0 %
1 Loss: 8.7269 Acc: 0.0000
0.0
best_acc 0

Epoch: 2
2 Loss: 16.2262 Acc: 0.0000
Accuracy for class: plane is 0.0 %
Accuracy for class: car   is 0.0 %
Accuracy for class: bird  is 0.0

In [30]:
base_encoder = ResNet50()

projection_dim=128
n_features= base_encoder.linear.in_features
net= SimCLR_model(base_encoder, projection_dim, n_features)

net= net.to(device)
#net

In [31]:

path_paramsend = "/content/gdrive/MyDrive/paramsend_bs_restart_r.pt"
checkpoint = torch.load(path_paramsend, map_location=torch.device('cpu'))
loss= checkpoint['loss']
epoch = checkpoint['epoch']

net.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [24]:
epoch

500

In [32]:
net_eval= base_encoder
n_features= base_encoder.linear.in_features
# for param in net_eval.parameters():
#     param.requires_grad = False
net_eval.fc=nn.Linear(n_features, len(classes)) #change the fc layer with linear layer
# net_eval.to(device)

In [33]:
start_epoch=0
best_acc = 0
batch_size = 32
lr = 0.0001
device='cuda'
optimizer= torch.optim.SGD(net_eval.parameters(), lr, momentum=0.9, weight_decay = 0, nesterov=True)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
net_eval.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (

In [34]:
net_eval.linear.weight #weights of the linear layer after loading the model 

Parameter containing:
tensor([[ 1.1506e-02,  3.7847e-05, -1.3727e-02,  ...,  1.5088e-03,
         -3.8930e-03, -2.3285e-03],
        [-1.3086e-02,  1.5437e-02, -1.1818e-02,  ...,  8.1926e-03,
          5.9451e-03, -1.3512e-02],
        [-1.6013e-02,  1.7593e-02,  1.2574e-02,  ..., -2.4490e-03,
         -1.1682e-03,  1.7180e-02],
        ...,
        [-5.9908e-03, -6.0457e-03, -1.0948e-02,  ..., -8.6522e-03,
          2.1111e-03, -1.1944e-02],
        [ 2.3940e-02, -9.7335e-03, -1.4288e-02,  ...,  3.5137e-03,
          1.6681e-02, -2.1242e-02],
        [-1.4279e-02, -1.4062e-02, -9.7422e-03,  ...,  1.2775e-02,
          2.8436e-03,  2.4568e-02]], device='cuda:0', requires_grad=True)

In [35]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

def train(epoch):
    print('\nEpoch: %d' % epoch)
    net_eval.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(evalloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net_eval(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(f'{epoch} Loss: {train_loss:.4f} Acc: {correct/total:.4f}')
best_acc=0
def test(epoch):
    global best_acc
    net_eval.eval()
    test_loss = 0
    correct = 0
    total = 0
    acc=0
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net_eval(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            for label, prediction in zip(targets, predicted):
              if label == prediction:
                  correct_pred[classes[label]] += 1
              total_pred[classes[label]] += 1


        # print accuracy for each class
        for classname, correct_count in correct_pred.items():
            accuracy = 100 * float(correct_count) / total_pred[classname]
            print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

        print(f'{epoch} Loss: {test_loss/(batch_idx+1):.4f} Acc: {correct/total:.4f}')

    acc = 100.*correct/total
    print(acc)
    if acc > best_acc:
        print('Saving..')
        parameters_finetune = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'loss': test_loss
        }

        # modelstate_save_name = 'modelstate_finetune.pt'
        # parameters_save_name = 'params_finetune.pt'
        # path_modelstate = F"/content/gdrive/MyDrive/{modelstate_save_name}" 
        # torch.save(net_eval.state_dict(), path_modelstate)
        # path_paramsfinetune = F"/content/gdrive/MyDrive/{parameters_save_name}" 
        # torch.save(parameters_finetune, path_paramsfinetune)
        best_acc = acc
        print("best accuracy", best_acc)

for epoch in range(start_epoch, start_epoch+30):
    train(epoch)
    test(epoch)

    # if epoch %25==0:
    #   parameters_end = {
    #         'net': net_eval.state_dict(),
    #         'acc': acc,
    #         'epoch': epoch,
    #         'loss': test_loss
    #   }
      # paramsend_save_name = 'paramsend_finetune.pt'
      # path_paramsend = F"/content/gdrive/MyDrive/{paramsend_save_name}" 
      # torch.save(parameters_end, path_paramsend)


Epoch: 0
0 Loss: 16.6754 Acc: 0.0000
Accuracy for class: plane is 0.0 %
Accuracy for class: car   is 0.0 %
Accuracy for class: bird  is 0.0 %
Accuracy for class: cat   is 0.0 %
Accuracy for class: deer  is 0.0 %
Accuracy for class: dog   is 0.0 %
Accuracy for class: frog  is 0.0 %
Accuracy for class: horse is 0.0 %
Accuracy for class: ship  is 0.0 %
Accuracy for class: truck is 0.0 %
0 Loss: 9.2074 Acc: 0.0000
0.0

Epoch: 1
1 Loss: 16.3038 Acc: 0.0000
Accuracy for class: plane is 0.0 %
Accuracy for class: car   is 0.0 %
Accuracy for class: bird  is 0.0 %
Accuracy for class: cat   is 0.0 %
Accuracy for class: deer  is 0.0 %
Accuracy for class: dog   is 0.0 %
Accuracy for class: frog  is 0.0 %
Accuracy for class: horse is 0.0 %
Accuracy for class: ship  is 0.0 %
Accuracy for class: truck is 0.0 %
1 Loss: 9.9118 Acc: 0.0000
0.0

Epoch: 2
2 Loss: 15.9000 Acc: 0.0000
Accuracy for class: plane is 0.0 %
Accuracy for class: car   is 0.0 %
Accuracy for class: bird  is 0.0 %
Accuracy for class: